In [18]:
import pandas as pd
import numpy as np
import re
import os

#### Extracting data from the Quotebank

In [ ]:
import re , math
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from scipy import sparse
nltk.download('punkt')
import bz2
import json
import pandas as pd
import re

In [8]:
#path to Quotebank file
path_to_file = 'data/Quotebank/quotes-2018.json.bz2'

#list with keywords for global warming
keywords_gw=['climatechange', 'global warming', 'fossil fuels', 'carbon dioxide', 'methane', 'co2', 'carbon neutral', 'sustainable', 'greenhouse gases', 'emission', 'sea-level rise', 'temperature', 'renewable energy', 'carbon footprint', 'deforestation', 'atmospheric aerosols', 'biofuel', 'cap and trade', 'Clean coal technology', 'Greenhouse effect', 'IPCC ', 'Kyoto Protocol', 'Major Economies Forum on Energy and Climate', 'Ocean acidification ', 'polution', 'UNFCCC ']
#list of keywords for abotion
keywords_ab = ['abortion', 'abortifacient', 'abortion on demand', 'abortion on request', 'age of consent', 'age of consent', 'Back-street abortion', 'Carrying a pregnancy to term', 'Conscientious objection', 'Fertilisation', 'Fetus', 'Gestation', 'Implantation', 'Induced abortion', 'Miscarriage', 'Pregnancy', 'Termination of pregnancy', 'abortion pill', 'abstinence', 'pro-life', 'pro-choice', 'anti-choice', 'borth control', 'Comstock Act', 'conception', 'contraception', 'contraceptive', 'Doe v. Bolton', 'embryo', 'Hyde Amendment', 'Intrauterine device', 'IUD', 'Planned Parenthood', 'medical abprtion', 'morning-after pill', 'misoprostol', 'Roe v. Wade', 'vacum aspiration abprtion', 'reproductive rights']
#list of keywords for gun rights
keywords_gun = ['gun rights', 'gun control', 'firearm owners protection act', 'title II', 'national firearm act', 'assault weapons ban', 'second amendment', 'national riffle association', 'march for our lives', 'gunowners of america', 'assault weapon', 'automatic weapon', 'semiautomatic weapon', 'Brady law', 'national instant criminal background check system', 'gun show loophole', 'strawman purchase', 'mass shootings', 'open carry', 'nonproliferation', 'gun law', 'the right of self-defence', 'gun crime', 'ban assault rifles', 'gun regulations', 'gun lobby', 'Washington DC vs Heller', 'Sandy Hook promise', 'gun license', 'gun ownership' ]
#list of keywords for gender ineqaulity
keywords_gen = ['gender inequality', 'gender discrimination', 'gender bias', 'gender blindness', 'gender equity', 'gender pay gap', 'unconscious biases', 'occupational segregation', 'wikigender', 'substantive equality', 'structural discrimination', 'structural barriers', 'patriarchy', 'parity education', 'gender stereotyping', 'gender-sensitive policies', 'gender roles', 'gender norms', 'gender mainstreaming', 'gender gap', 'gender equity', 'women empowerment', 'sexism', 'misogyny', 'feminism', 'misandry', 'title IX', 'male gaze', 'bropropriating', 'mansplain', 'manterrupting', 'toxic masculinity', 'male privilege']

In [ ]:
def process_chunk(chunk,keywords):
    #logging
    print(f'Processing chunk with {len(chunk)} rows')
    #reurn only rows containing key words
    return chunk[chunk['quotation'].apply(lambda x:any(key in ' '.join([wnl.lemmatize(words) for words in nltk.word_tokenize(re.sub('[^A-Za-z0-9\s]+', '', x.lower()))]) for key in keywords))]

def extract(keywords)
    wnl = WordNetLemmatizer()

    #lemmatization
    keywords_lm=[' '.join(words) for words in [nltk.word_tokenize(key) for key in keywords]]
    
    #reading big file by chunks
    df_reader =pd.read_json(path_to_file, lines=True, compression='bz2', chunksize=500000)
    k=0
    for chunk in df_reader:
        gl_warm=process_chunk(chunk,keywords_lm).copy()
        gl_warm.to_csv('global_warming_2018'+'_'+str(k)+'.csv')
        k+=1

extract(keywords_gw)

Concatenating files after extracting

In [6]:
def concat(dir_in,file_out):
    df=pd.DataFrame()
    for file in os.listdir(dir_in):
        if file[-3:]=='csv':
            df_cur=pd.read_csv(dir_in+'/'+file,index_col='Unnamed: 0')
            df=pd.concat([df,df_cur]).copy()
    df.to_csv(file_out)
concat('out/2017','data/global_warming/global_warming_2017.csv')

#### Merging and cleaning all the years into one file with speakers' attributes

In [19]:
def clean_df(path):
    d=pd.read_csv(path,index_col='Unnamed: 0')
    d['probas']=d['probas'].apply(lambda x: re.sub('[\[\]\'\s]','',x).split(','))
    d['pr']=d['probas'].apply(lambda x: [x[i] for i in range(len(x)) if i%2==1])
    d['au_pr']=d['probas'].apply(lambda x: [x[i] for i in range(len(x)) if i%2==0])
    d=d[d['pr'].apply(lambda x: all([len(x[i])>1  for i in range(len(x))]))].copy() #deleting empty probas
    d=d[d['pr'].apply(lambda x: all([x[i][0]=='0' for i in range(len(x))]))].copy() #deleting wrong parsed probas
    d=d[d['pr'].apply(lambda x: float(x[0])-float(x[1])>0.3)] #deleting uncertained authors
    d=d[d['speaker']!='None'].copy() #deleting None author
    df_out=d[['quotation','speaker','qids','numOccurrences','urls']] #leaving only usefull columns
    print('Size of the cleaned dataframe is',df_out.shape)
    return df_out

In [23]:
def merge_attributes(df_out):
    #merging with information about speakers
    dff=pd.DataFrame()
    for i in range(0,16):
        if i<10:
            s='0'+str(i)
        else:
            s=str(i)
        df=pd.read_parquet('data/speaker_attributes/part-000'+s+'-0d587965-3d8f-41ce-9771-5b8c9024dce9-c000.snappy.parquet')
        df['year_of_birth']=df['date_of_birth'].apply(lambda x: x[0].split('-')[0].strip('+') if x is not None else x)
        dff=pd.concat([dff,df_out.merge(df[['gender', 'nationality', 'ethnic_group', 'academic_degree', 'religion','label','year_of_birth']],how='left',left_on='speaker',right_on='label').dropna(subset=['gender'])]).copy()
    #getting final dataframe
    dff=dff.drop_duplicates(subset=['speaker','quotation']).copy()
    dff=dff.drop(columns='label').copy()
    print('Size of the dataframe with information about authors is',dff.shape)
    return dff

In [24]:
def final_df(dir):
    final=pd.DataFrame()
    for file in os.listdir(dir):
        if file[-3:]=='csv':
            df_out=clean_df(os.path.join(dir,file))
            dff=merge_attributes(df_out)
            dff['year']=file.split('_')[-1].split('.')[0]
            final=pd.concat([final,dff]).copy()
    return final

In [25]:
final_df('data/global_warming').to_csv('data/attributes/global_warming.csv')

Size of the cleaned dataframe is (23085, 5)
Size of the dataframe with information about authors is (21426, 11)
Size of the cleaned dataframe is (11613, 5)
Size of the dataframe with information about authors is (10858, 11)
Size of the cleaned dataframe is (14562, 5)
Size of the dataframe with information about authors is (13531, 11)
Size of the cleaned dataframe is (16652, 5)
Size of the dataframe with information about authors is (15439, 11)
Size of the cleaned dataframe is (21494, 5)
Size of the dataframe with information about authors is (19880, 11)
Size of the cleaned dataframe is (13089, 5)
Size of the dataframe with information about authors is (12167, 11)


In [26]:
final_df('data/abortion').to_csv('data/attributes/abortion.csv')

Size of the cleaned dataframe is (8611, 5)
Size of the dataframe with information about authors is (8085, 11)
Size of the cleaned dataframe is (1636, 5)
Size of the dataframe with information about authors is (1510, 11)
Size of the cleaned dataframe is (1610, 5)
Size of the dataframe with information about authors is (1442, 11)
Size of the cleaned dataframe is (1070, 5)
Size of the dataframe with information about authors is (1001, 11)
Size of the cleaned dataframe is (1362, 5)
Size of the dataframe with information about authors is (1263, 11)
Size of the cleaned dataframe is (1940, 5)
Size of the dataframe with information about authors is (1828, 11)


In [27]:
final_df('data/gun_control').to_csv('data/attributes/gun_control.csv')

Size of the cleaned dataframe is (1834, 5)
Size of the dataframe with information about authors is (1679, 11)
Size of the cleaned dataframe is (1639, 5)
Size of the dataframe with information about authors is (1538, 11)
Size of the cleaned dataframe is (1442, 5)
Size of the dataframe with information about authors is (1336, 11)
Size of the cleaned dataframe is (1919, 5)
Size of the dataframe with information about authors is (1782, 11)
Size of the cleaned dataframe is (1568, 5)
Size of the dataframe with information about authors is (1444, 11)
Size of the cleaned dataframe is (816, 5)
Size of the dataframe with information about authors is (738, 11)


In [28]:
final_df('data/gender_inequality').to_csv('data/attributes/gender_inequality.csv')

Size of the cleaned dataframe is (1483, 5)
Size of the dataframe with information about authors is (1410, 11)
Size of the cleaned dataframe is (2445, 5)
Size of the dataframe with information about authors is (2303, 11)
Size of the cleaned dataframe is (3235, 5)
Size of the dataframe with information about authors is (3064, 11)
Size of the cleaned dataframe is (6079, 5)
Size of the dataframe with information about authors is (5715, 11)
Size of the cleaned dataframe is (1030, 5)
Size of the dataframe with information about authors is (974, 11)
Size of the cleaned dataframe is (833, 5)
Size of the dataframe with information about authors is (782, 11)
